# Understanding AI Agents: A Simple Guide to LangChain, AutoGen, LangGraph, and LangGraph Studio


## Setup and Installation

First, let's install the necessary packages:

In [ ]:
# Install required packages
!pip install langchain langchain_openai wikipedia pyautogen langgraph

## LangChain Agent Framework

[LangChain](https://www.langchain.com/) provides a popular framework for building LLM-powered agents.

### Key Concepts in LangChain

1. **Agents**: The central entity that decides what actions to take
2. **Tools**: Functions that agents can call to interact with the world
3. **Memory**: Storage mechanisms for preserving context
4. **Chains**: Sequences of operations that can be composed together

### Simple LangChain Agent Example with Wikipedia Search

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

# Set up Wikipedia search tool
wikipedia = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia)

# Create the tool properly as a Tool object, not a dictionary
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia_tool.run,
        description="Useful for searching Wikipedia for information"
    )
]

# Initialize the LLM
llm = OpenAI(temperature=0)

# Create the prompt template for the agent with agent_scratchpad included
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent
agent_executor.invoke({"input": "Where is Northeastern university located? What are the research programs they offer?"})



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to search for information about Northeastern university and their research programs.
Action: Wikipedia
Action Input: 'Northeastern University'Page: Northeastern University
Summary: Northeastern University (NU or NEU) is a private research university with its main campus in Boston, Massachusetts, United States. It was founded by the Boston Young Men's Christian Association in 1898 as an all-male institute before being incorporated as Northeastern College in 1916, gaining university status in 1922. With more than 38,000 students, Northeastern is the largest university in Massachusetts by enrollment.
Northeastern is a large, highly residential university which is composed of ten colleges, including the Northeastern University School of Law. The university's main campus in Boston is located within the center of the city along Huntington Avenue and Columbus Avenue near the Fenway–Kenmore and Roxbury neighborhoods. It of

{'input': 'Where is Northeastern university located? What are the research programs they offer?',
 'output': 'Northeastern University is located in Boston, Massachusetts, but also has satellite campuses in Charlotte, Seattle, San Jose, Oakland, Portland, Burlington, Miami, London, and Toronto and Vancouver in Canada. They offer undergraduate and graduate programs, and are known for their cooperative education program. They also have a campus in Shenyang, China and a campus in London, England, which was originally founded as New College of the Humanities and later acquired by Northeastern University.'}

When this code runs:
1. The agent receives a query
2. It decides to use the Wikipedia tool to look up information about Paris
3. It receives results and formulates an answer

## AutoGen Framework

[AutoGen](https://microsoft.github.io/autogen/) (developed by Microsoft) focuses on enabling multiple agents to collaborate with each other.

### Key Concepts in AutoGen

1. **Multi-Agent Conversations**: Agents can talk to each other to solve problems
2. **Specialized Agents**: Different agents can have different roles (coder, reviewer, etc.)
3. **Human-in-the-loop**: Humans can participate in agent conversations

### Simple AutoGen Example with Wikipedia Search

In [ ]:
import json
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import wikipedia  # pip install wikipedia

# Create a configuration file for API keys
config_list = [
    {
        'model': 'gpt-4o',
        'api_key': os.environ["OPENAI_API_KEY"]
    }
]

# Write the configuration to a JSON file
with open('OAI_CONFIG_LIST.json', 'w') as f:
    json.dump(config_list, f)

# Configure agents
config_list = config_list_from_json("OAI_CONFIG_LIST.json")

# Create a Wikipedia search function
def search_wikipedia(query):
    try:
        # Search Wikipedia
        search_results = wikipedia.search(query)

        if not search_results:
            return "No results found on Wikipedia."

        # Get the page for the first result
        page = wikipedia.page(search_results[0])

        # Get a summary
        summary = page.summary

        return f"Title: {page.title}\nSummary: {summary}\nURL: {page.url}"
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

# Function definitions for the LLM in OpenAI format
function_definitions = [
    {
        "name": "search_wikipedia",
        "description": "Search Wikipedia for information on a topic",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The topic to search for on Wikipedia"
                }
            },
            "required": ["query"]
        }
    }
]

# Create the assistant agent with function calling capabilities
assistant = AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,
        "functions": function_definitions
    },
    # Add termination keywords that the assistant should recognize
    system_message="You are a helpful assistant. After answering the user's question comprehensively, say 'TERMINATE' to end the conversation."
)

# Create the user proxy agent with function execution capability
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",  # Never ask for human input
    function_map={"search_wikipedia": search_wikipedia},
    # Add termination criteria
    code_execution_config=False,  # Disable code execution prompts
    default_auto_reply="TERMINATE"  # Automatically terminate when receiving a complete answer
)

# Start the conversation with a specific question and get the result
result = user_proxy.initiate_chat(
    assistant,
    message="What is the Eiffel Tower and when was it built?"
)

print("\nConversation completed automatically.")

user_proxy (to assistant):

What is the Eiffel Tower and when was it built?

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: search_wikipedia *****
Arguments: 
{"query":"Eiffel Tower"}
*****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_wikipedia...
Call ID: None
Input arguments: {'query': 'Eiffel Tower'}
user_proxy (to assistant):

***** Response from calling function (search_wikipedia) *****
Title: Eiffel Tower
Summary: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's F

In this example:
1. We create three agents: an assistant, a Wikipedia agent, and a user proxy
2. The Wikipedia agent provides a search tool
3. The assistant can use this tool to answer questions
4. The agents work together to produce a response

## LangGraph Framework

[LangGraph](https://github.com/langchain-ai/langgraph) (built on top of LangChain) provides a way to create stateful, multi-step agent workflows using a graph structure.

### Key Concepts in LangGraph

1. **Nodes**: Individual steps in the agent's workflow
2. **Edges**: Connections between nodes that determine execution flow
3. **State**: Information that persists throughout the agent's execution
4. **Conditional Logic**: The ability to follow different paths based on certain conditions

### Simple LangGraph Example with Wikipedia Search

In [ ]:
from langgraph.graph import StateGraph
from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from typing import TypedDict, Annotated
from IPython.display import display, Markdown, HTML

# Define state schema
class State(TypedDict):
    query: str
    search_results: Annotated[str, None]
    response: Annotated[str, None]

# Set up Wikipedia search tool
wikipedia = WikipediaAPIWrapper()
search_tool = WikipediaQueryRun(api_wrapper=wikipedia)

# Define nodes
def search_wikipedia(state: State) -> State:
    """Search Wikipedia for information related to the query"""
    query = state["query"]
    results = search_tool.run(query)

    # Display intermediate search results
    display(HTML("<h3>Wikipedia Search Results:</h3>"))
    display(Markdown(results[:500] + "..." if len(results) > 500 else results))

    return {"search_results": results}

def generate_response(state: State) -> State:
    """Generate a response based on the search results"""
    llm = OpenAI(temperature=0.7)
    prompt = ChatPromptTemplate.from_template(
        "Based on the following information: {search_results}, "
        "please answer the question: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"search_results": state["search_results"], "query": state["query"]})

    # Check if response is a string or an object with content attribute
    if hasattr(response, 'content'):
        final_response = response.content
    else:
        # If it's already a string, use it directly
        final_response = response

    return {"response": final_response}

# Create the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("search", search_wikipedia)
workflow.add_node("respond", generate_response)

# Add edges
workflow.set_entry_point("search")
workflow.add_edge("search", "respond")
workflow.set_finish_point("respond")

# Compile the graph
app = workflow.compile()

# Execute the query and display the result nicely
query = "Who invented the telephone?"
print(f"Searching for: {query}")

# Run the graph
results = app.invoke({"query": query})

# Display the final response in a nice format
display(HTML("<h2>Final Answer:</h2>"))
display(Markdown(results["response"]))

Searching for: Who invented the telephone?


Page: Telephone
Summary: A telephone, colloquially referred to as a phone, is a telecommunications device that enables two or more users to conduct a conversation when they are too far apart to be easily heard directly. A telephone converts sound, typically and most efficiently the human voice, into electronic signals that are transmitted via cables and other communication channels to another telephone which reproduces the sound to the receiving user. The term is derived from Ancient Greek: τῆλε...



Alexander Graham Bell is credited with inventing the telephone.

In this example:
1. We define two nodes: one for searching Wikipedia and one for generating a response
2. We create a graph with these nodes
3. We define the flow: search first, then respond
4. When executed, the graph maintains state between nodes

## LangGraph Studio

[LangGraph Studio](https://langchain-ai.github.io/langgraph-studio/) is a visual development environment for building and debugging LangGraph applications.

### Key Features of LangGraph Studio

1. **Visual Graph Builder**: Create agent workflows using a drag-and-drop interface
2. **Real-time Testing**: Test your agent workflows as you build them
3. **State Inspection**: Examine the state of your agent at each step
4. **Debugging Tools**: Identify and fix issues in your agent workflows

### Setting Up LangGraph Studio

LangGraph Studio is typically run as a separate application. Here's how to install and run it:

In [ ]:
# Install LangGraph Studio
!pip install langgraph-studio

# NOTE: To run LangGraph Studio, execute the following in your terminal (not in this notebook):
# python -m langgraph_studio

## Saving a LangGraph for LangGraph Studio

Here's how to save a LangGraph so it can be loaded in LangGraph Studio:

In [ ]:
# Save the workflow for LangGraph Studio
import pickle

# Create a simple Wikipedia search graph
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated

# Define state schema
class WikiSearchState(TypedDict):
    query: str
    search_results: Annotated[str, None]
    response: Annotated[str, None]

# Create a simple graph
wiki_graph = StateGraph(WikiSearchState)
wiki_graph.add_node("search", search_wikipedia)
wiki_graph.add_node("respond", generate_response)
wiki_graph.set_entry_point("search")
wiki_graph.add_edge("search", "respond")
wiki_graph.set_finish_point("respond")

# Save the graph definition
with open("wiki_search_graph.pkl", "wb") as f:
    pickle.dump(wiki_graph, f)

print("Saved graph for LangGraph Studio. Load it with 'python -m langgraph_studio wiki_search_graph.pkl'")

## Comparing the Frameworks

| Framework | Key Strength | Best Use Case |
|-----------|--------------|---------------|
| LangChain | Simplicity and flexibility | Single agents with multiple tools |
| AutoGen | Multi-agent collaboration | Complex problems requiring different expertise |
| LangGraph | Complex, stateful workflows | Multi-step reasoning with branching logic |
| LangGraph Studio | Visual development | Rapid prototyping and debugging |

## Conclusion

AI agents represent a powerful paradigm for extending the capabilities of LLMs. By understanding these frameworks, you can choose the right approach for your specific application:

- Use **LangChain** when you need a simple, flexible agent with access to tools
- Use **AutoGen** when you need multiple agents to collaborate
- Use **LangGraph** when you need complex, multi-step workflows with conditional logic
- Use **LangGraph Studio** when you want to build and debug your agent workflows visually

Each framework offers different trade-offs in terms of simplicity, flexibility, and power, but they all share the common goal of making LLMs more capable by giving them agency in the world.